# Librerias

In [ ]:
!pip install scrapy
!scrapy startproject bookscraper
%cd bookscraper/bookscraper/spiders/

# Funciones

# Web Scrapping

In [2]:
!scrapy genspider WikiSpider es.wikipedia.org/wiki/Programaci%C3%B3n

Created spider 'WikiSpider' using template 'basic' 


In [ ]:
!scrapy runspider WikiSpider.py -O ../../ejercicio0.json

# Pre procesamiento

# Vectorizacion

# Base de Datos

# Consulta